In [1]:
# Фикс: в новых версиях Python inspect.getargspec отсутствует
import inspect
from collections import namedtuple

if not hasattr(inspect, "getargspec"):
    ArgSpec = namedtuple('ArgSpec', 'args varargs keywords defaults')
    def getargspec(func):
        spec = inspect.getfullargspec(func)
        return ArgSpec(spec.args, spec.varargs, spec.varkw, spec.defaults)
    inspect.getargspec = getargspec

# Основные библиотеки
import pandas as pd   # для работы с таблицами
import math           # для логарифмов и мат. операций
import pymorphy2      # морфологический анализатор для русского
from transformers import AutoTokenizer  # WordPiece токенизатор
import re             # регулярные выражения


d:\AvitoTask\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Инициализация инструментов
morph = pymorphy2.MorphAnalyzer()  # морфологический анализатор для распознавания слов и форм
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")  # компактный WordPiece токенизатор

print("MorphAnalyzer и токенизатор загружены")


d:\AvitoTask\.venv\Lib\site-packages\pymorphy2\analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


MorphAnalyzer и токенизатор загружены


In [3]:
# Загрузка частотного словаря ru_full.txt
word_freqs = {}
with open("ru_full.txt", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) == 2:
            word, freq = parts
            word_freqs[word.lower()] = int(freq)

# Преобразуем частоты в логарифмы, чтобы снизить большие различия
log_freqs = {w: math.log(f) for w, f in word_freqs.items()}
print(f"Загружено {len(word_freqs)} слов в словарь")


Загружено 1423050 слов в словарь


In [4]:
def get_word_score(word: str) -> float:
    """
    Оценивает слово по pymorphy2 и частотному словарю
    Чем выше score, тем более правдоподобно, что сегмент является словом
    """
    # Наказываем слишком короткие сегменты, чтобы избежать чрезмерного дробления текста
    if len(word) < 2:
        return -1.2  
    elif len(word) < 3:
        return -0.8  
    elif len(word) < 4:
        return -0.3  

    parses = morph.parse(word)
    score = 0.0

    # Учитываем морфологический разбор pymorphy2
    if parses:
        best = parses[0]  # берём наиболее вероятный разбор
        prob = best.score if best.score > 0 else 1e-6
        score += math.log(prob + 1e-9)

    # Учитываем частотный словарь: часто встречающиеся слова получают бонус
    if word.lower() in log_freqs:
        score += log_freqs[word.lower()]

    return score


In [5]:
def segment(text: str):
    """
    Делит текст на слова с помощью динамического программирования
    Ищем разбиение с максимальной суммарной оценкой (get_word_score)
    """
    n = len(text)
    # dp[i] = (лучший score до позиции i, индекс начала последнего слова)
    dp = [(-1e12, -1)] * (n + 1)
    dp[0] = (0.0, -1)  # стартовое условие

    # Основной цикл: для каждой позиции ищем лучшее разбиение
    for i in range(1, n + 1):
        best_score, best_j = -1e12, -1
        # Ограничиваем длину слова (макс. 24 символа) для скорости
        for j in range(max(0, i - 24), i):
            word = text[j:i]
            score = dp[j][0] + get_word_score(word)
            if score > best_score:
                best_score, best_j = score, j
        dp[i] = (best_score, best_j)

    # Восстанавливаем слова (идём назад от конца текста)
    segments = []
    i = n
    while i > 0:
        j = dp[i][1]
        segments.append((j, i))  # добавляем границу сегмента
        i = j
    segments.reverse()

    return segments


In [6]:
def heuristic_positions(text):
    """
    Простая эвристика: ищет подряд идущие цифры или латинские буквы
    После них вставляет пробел (позиция = конец последовательности)
    """
    positions = []
    for match in re.finditer(r"[0-9a-zA-Z]+", text):
        positions.append(match.end())  # конец найденного блока
    return positions


In [7]:
def get_predicted_positions(text):
    """
    Основная функция для предсказания позиций пробелов.
    Комбинирует:
    - динамическое программирование (segment)
    - частотный словарь + pymorphy2
    - WordPiece для редких/длинных слов
    - эвристики (цифры/латиница, заглавные буквы)
    """
    segments = segment(text)
    positions = []

    for start, end in segments:
        word = text[start:end]
        parses = morph.parse(word)

        # если слово узнаётся морфологическим анализатором или оно короткое, то ставим пробел после него
        if parses or len(word) <= 8:
            if len(word) >= 1: # тут изначально ставил ограничения, позже исправил штрафами
                positions.append(end)
        else:
            # для длинных неизвестных слов применяем WordPiece разбиение
            tokens = tokenizer.tokenize(word)
            idx = start
            for tok in tokens[:-1]:  # последний токен не даёт пробел
                clean_tok = tok.replace("##", "")
                idx += len(clean_tok)
                positions.append(idx)
            positions.append(end)

    # эвристика для цифр и латиницы
    positions += heuristic_positions(text)

    # эвристика: пробел перед одиночной заглавной буквой или новым словом с заглавной
    for i in range(1, len(text)):
        if text[i].isupper() and text[i-1].islower():
            positions.append(i)

    # удаляем дубликаты и сортируем
    positions = sorted(list(set(positions)))

    return positions


In [8]:
rows = []
with open("dataset_1937770_3.txt", "r", encoding="utf-8") as f:
    next(f)  # пропускаем заголовок
    for line in f:
        line = line.rstrip("\n")
        if not line:
            continue  # пропускаем пустые строки
        parts = line.split(",", 1)
        if len(parts) == 2:
            rows.append(parts)  # нормальная строка с id и текстом
        else:
            rows.append([parts[0], ""])  # если текста нет, подставляем пустую строку

# создаём DataFrame для дальнейшей обработки
task_data = pd.DataFrame(rows, columns=["id", "text_no_spaces"])
print(f"Загружено {len(task_data)} строк")


Загружено 1005 строк


In [9]:
pred_positions_list = []
for text in task_data['text_no_spaces']:
    # если пустая строка или не строка, то добавляем пустой список
    if not isinstance(text, str) or len(text.strip()) == 0:
        pred_positions_list.append("[]")
    else:
        # получаем позиции вставки пробелов
        positions = get_predicted_positions(text)
        # убираем последний элемент (чтобы не считать конец строки как часть разбиения)
        pred_positions_list.append(str(positions[:-1]))

# добавляем колонку с предсказанными позициями
task_data['predicted_positions'] = pred_positions_list
print("predicted_positions сформированы")


predicted_positions сформированы


In [10]:
# сохраняем результат в submission.csv для отправки на проверку
task_data.to_csv("submission.csv", index=False)
print("submission.csv готов")


submission.csv готов
